In [1]:
%matplotlib inline
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
import feature_effect as fe
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
plt.rcParams["figure.figsize"] = (15,10)

### Example

- $f(x_1, x_2) = x_1^2 + x_1^2x_2$
- $p(x_1, x_2) = \mathcal{U}(x_1; 0,1)\mathcal{N}(x_2; 0,1)$
- $\frac{\partial f}{\partial x_1} = 2x_1(1 + x_2)$
- $\frac{\partial f}{\partial x_2} = x_1^2$

In [2]:
def generate_samples(N, seed):
    if seed is not None:
        np.random.seed(seed)

    x1 = np.random.uniform(size=N)
    x2 = np.random.normal(size=N)
    return np.stack([x1, x2]).T


def f(x):
    return x[:,0]**2 + x[:,0]**2 * x[:,1]


def f_der(x):
    return np.stack([2*x[:,0]*(1 + x[:,1]), x[:,0]**2], axis=-1)

### Feature effects
- $f_{ALE}(x_1) = x_1^2$
- $f_{ALE}(x_2) = \frac{x_2}{3}$

##### Proofs

- $f_{ALE}(x_1) = \int_{0}^{x_1} \mathbb{E}_{x_2|x_1=z} [\frac{\partial f}{\partial x_1}] \partial z = \int_{0}^{x_1} \mathbb{E}_{x_2} [2z(1 + x_2)] \partial z = \int_{0}^{x_1} 2z (1 + \mathbb{E}_{x_2}[x_2]) \partial z = 2 \int_{0}^{x_1} z \partial z = x_1^2$
- $f_{ALE}(x_2) = \int_{0}^{x_2} \mathbb{E}_{x_1|x_2=z} [\frac{\partial f}{\partial x_2}] \partial z = \int_{0}^{x_2} \mathbb{E}_{x_1} [x_1^2] \partial z = \frac{1}{3}\int_{0}^{x_2}  \partial z = \frac{x_2}{3}$

In [3]:
def f_ale_x1(x1):
    z = 1/3
    return x1**2 - z

def f_ale_x2(x2):
    z = 0
    return x2/3 - z

#### Generate samples

In [4]:
seed = 21
N = 100
X = generate_samples(N, seed)
y = f(X)
dy = f_der(X)

# plt.figure()
# plt.plot(X[:,0], X[:,1], "ro")
# plt.show(block=False)

In [5]:
#### Define feature effect methods

In [6]:
ale = fe.ALE(data=X, model=f)
dale = fe.DALE(data=X, model=f, model_jac=f_der)

#### Feature 1

In [7]:
def plot(bins):
    ale.fit(features=[0, 1], k=bins)
    ale.plot(s=0, block=False, gt=f_ale_x1)
    plt.show()
    
interactive_plot = widgets.interactive(plot, bins=(1, 100, 1))
output = interactive_plot.children[-1]
output.layout.height = '700px'
interactive_plot

interactive(children=(IntSlider(value=50, description='bins', min=1), Output(layout=Layout(height='700px'))), …

In [8]:
def plot(bins):
    dale.fit(features=[0, 1], k=bins)
    dale.plot(s=0, block=False, gt=f_ale_x1)
    plt.show()

dale_plot = widgets.interactive(plot, bins=(1, 100, 1))
dale_output = interactive_plot.children[-1]
dale_output.layout.height = '350px'
dale_plot

interactive(children=(IntSlider(value=50, description='bins', min=1), Output()), _dom_classes=('widget-interac…

#### Feature 2

In [9]:
def plot(bins):
    ale.fit(features=[0, 1], k=bins)
    ale.plot(s=1, block=False, gt=f_ale_x2)
    plt.show()
    
interactive_plot = widgets.interactive(plot, bins=(1, 100, 1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=50, description='bins', min=1), Output(layout=Layout(height='350px'))), …

In [10]:
dale = fe.DALE(data=X, model=f, model_jac=f_der)
def plot(bins):
    dale.fit(features=[0, 1], k=bins)
    dale.plot(s=1, block=False, gt=f_ale_x2)
    plt.show()

interactive_plot = widgets.interactive(plot, bins=(1, 100, 1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=50, description='bins', min=1), Output(layout=Layout(height='350px'))), …

#### Generic Estimator

In [11]:
est = fe.Estimator(data=X, model=f, model_jac=f_der)

def plot(bins):
    est.fit(features=[0,1], method="all", nof_bins=bins)
    est.plot(feature=0, method="all", ale_gt=f_ale_x1)
    plt.show()

interactive_plot = widgets.interactive(plot, bins=(1, 100, 1))
output = interactive_plot.children[-1]
output.layout.height = '800px'
interactive_plot

interactive(children=(IntSlider(value=50, description='bins', min=1), Output(layout=Layout(height='800px'))), …